# Colab VM configuration information

This notebook is intended simply for dumping status information from a Colab runtime (kernel, VM, whatever).

In [ ]:
# Various installs of packages imported by colab_vm_config_info.ipynb
!pip install humanize
!pip install gputil
!pip install psutil

## Python environment stats


In [ ]:
# What version of Python is in effect?

import platform

a_message = "Python runtime version: " + platform.python_version() 
print(a_message)

In [ ]:
# What packages are installed for the detected running version of Python?
import platform

python_major_version = int(platform.python_version_tuple()[0])
print(python_major_version)

if python_major_version == 3:
  print("Python 3.6 dist-packages")
  !ls /usr/local/lib/python3.6/dist-packages
else: 
  # Python 2 it is...
  print("Python 2.7 dist-packages")
  !ls /usr/local/lib/python2.7/dist-packages

## Jupyter stats


In [ ]:
!jupyter-kernelspec list

## Check VM Uptime

Supposedly these Colab kernel (runtimes, VMs, whatever) are thrown away after a max of 12 hours. In the results of `uptime` is the time-of-day (formatted `HH:MM:SS`) followed by ` up: ` followed by how long the VM has been up (formatted as `XX min`, or after an hour as `H:MM`).

Note, the 12 hour lifetime cap is the max but less can happen it seems. Seemingly, [this issue is not well defined by google](https://stackoverflow.com/questions/55050988/can-i-run-a-google-colab-free-edition-script-and-then-shutdown-my-computer).

Furthermore, VM max lifetime is different than the constant runtime disconnects. These seem to happen on the order of 60 to 90 minutes. Supposedly keeping a code cell running can keep the connection from disconnect timeouts. Note though that editing text cells does not reset the timeout clock.

In [ ]:
# How long has this VM been up? 
!uptime

## Reset VM

In [ ]:
# Reset VM
!kill -9 -1

## CPU Information

In [ ]:
# How many CPUs does this machine have?
!lscpu | grep "^CPU(s):"

In [ ]:
# More details on the CPU(s):
!cat /proc/cpuinfo

In [ ]:
# RAM
!cat /proc/meminfo | head -n3

In [ ]:
# RAM info humanized
import psutil
import humanize
import os

process = psutil.Process(os.getpid())
print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))


## File System

As with AWS Lambda, there are intentionally few switches for selecting VM options. Memory and CPU are provided as matching packages, not independently configurable.

So, depending on what you ask for in terms of compute (CPU, GPU, [TPU](https://colab.research.google.com/notebooks/tpu.ipynb)) you get more or less file system memory [[*](https://stackoverflow.com/a/55890688)]. Note: for all compute options, the OS files initialize to consuming about 25MB of the file system before you are dropped into the kernel. 

On 2019-05-19, the following tests gave these results:

Processor | FS Free GB | FS Total GB 
--|--|--
CPU | 24 | 49
GPU | 318 | 359
TPU | 26| 49

The low FS size for the TPU is probably because for the TPU case, those (the actually TPU boards) are separate machines while for GPUs those  are a part of the machine the notebook is running on. So, for the CPU and the TPU options, Google is probably providing the same VM, ergo the file systems are essentially the same size. 

In [ ]:
!df -h .

## GPU Information

In [ ]:
# What GPU is currently config'd for use?

# https://colab.research.google.com/notebooks/gpu.ipynb
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print("GPU detected: NONE")
else:
  print('GPU detected: {}'.format(device_name))

# TODO: Another way?
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

In [ ]:
# Memory in GPU
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi

import GPUtil as GPU

gpus = GPU.getGPUs()

if len(gpus) > 0:
  gpu = gpus[0]
  print("GPU RAM:\n Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
else:
  print("GPU detected: NONE")

In [ ]:
!nvcc --version
!conda install tsnecuda cuda100 -c cannylab

## References

* [Google Colab Free GPU Tutorial](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d)
* [GPU stats code](https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available)
* [TensorFlow with GPU](https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=3IEVK-KFxi5Z)